In [ ]:
'''
@Author: Yitao Qiu
'''
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

fix_seed = 2021
torch.manual_seed(fix_seed)
torch.cuda.manual_seed_all(fix_seed)
np.random.seed(fix_seed)
torch.backends.cudnn.deterministic = True
from utils.tools import dotdict
#parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

In [ ]:
os.path.exists('/hy-tmp/crypto.csv')

In [ ]:
args = dotdict()
args.is_training=True
args.model_id='LOB'
args.model = 'Transformer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]


args.data = 'custom' # data
args.root_path = '/hy-tmp/' # root path of data file
args.data_path = 'crypto.csv' # data file


args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'mid_price' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[us,microsecond,s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './checkpoint' # location of model checkpoints

args.seq_len = 100 # input sequence length of Informer encoder
args.label_len = 50 # start token length of Informer decoder
args.pred_len = 100 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]
args.bucket_size= 4 #'for Reformer'
args.n_hashes= 4 #'for Reformer'

# LSTM
args.hidden_size = 64
args.num_layers = 2


args.individual = False

# FEDformer
args.version = 'Fourier'
args.mode_select = 'random'
args.modes = 64
args.L = 3
args.base = 'legendre'
args.cross_activation = 'tanh'
args.embed_type = 0

args.enc_in = 41 # encoder input size
args.dec_in = 41 # decoder input size
args.c_out = 1 # output size
args.factor = 3 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.moving_avg=25 #'window size of moving average'
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
#args.freq = 'h'
args.do_predict =False
args.batch_size = 256 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 10
args.itr = 1
args.train_epochs = 10
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [ ]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [ ]:
print('Args in experiment:')
print(args)

In [ ]:
Exp = Exp_Main

In [ ]:
# exp = Exp(args)

In [ ]:
# train_data, train_loader = exp._get_data(flag='train')

In [ ]:
# seq_x, seq_y, seq_x_mark, seq_y_mark = train_data.__getitem__(0)

In [ ]:
# seq_x

In [ ]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_sl{}_ll{}_pl{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.seq_len,
            args.label_len,
            args.pred_len,
            ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)

        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_sl{}_ll{}_pl{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.seq_len,
            args.label_len,
            args.pred_len,
            ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()
